In [1]:
import requests  
import os
import re
import time
import json
import random
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pandas import DataFrame
import pymysql

In [22]:
start_path='2008a/hebei'
all_path=os.listdir(start_path)

In [23]:
len(all_path)

356874

In [ ]:
def dbHandle():
    conn = pymysql.connect(
        host='localhost',
        user='root',
        passwd='9432',
        charset='utf8',
        use_unicode=False
    )
    return conn
dbObject = dbHandle()
cursor = dbObject.cursor()
sql = 'insert into company.hebei(url,company,province,website,industry_b,headquarters,welfare,工商注册号,组织机构代码,统一信用代码,经营状态,行业,企业类型,法定代表人,营业期限,注册资本,核准日期,登记机关,企业地址,经营范围,企业联系电话,电子邮箱,主要人员,股东信息,热招职位) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

n=0
while n<len(all_path):
    print(n)
    with open(start_path+'/'+all_path[n],'r',encoding='utf-8') as f:
        soup=BeautifulSoup(f.read(),'lxml')
        if not soup.find('span',class_="title line-clamp1"):
            print(all_path[n],'空')
            n+=1
            continue
        item={}
        item["url"]=''
        item["company"]=''
        item["website"]=''
        #item["province"]=''
        item["headquarters"]=''
        item["industry_b"]=''
        item["welfare"]=''
        item['工商注册号']=''
        item['组织机构代码']=''
        item['统一信用代码']=''
        item['经营状态']=''
        item['行业']=''
        item['企业类型']=''
        item['法定代表人']=''
        item['营业期限']=''
        item['注册资本']=''
        item['核准日期']=''
        item['登记机关']=''
        item['企业地址']=''
        item['经营范围']=''
        item['企业联系电话']=''
        item['电子邮箱']=''
        item['主要人员']=''
        item['股东信息']=''
        item['热招职位']=''
        
        
        item["province"]='hebei'
        
        if soup.find('a',target="_blank").text!='+关注':
            item["website"]=soup.find('a',target="_blank").text
        #item["url"]=response.url
        item["company"]=soup.find('span',class_="title line-clamp1").text.strip()
        
        t=soup.find('div',class_="base").text
        if re.search('总部地点',t):
            item["headquarters"]=t[re.search('总部地点',t).end()+1:re.search('总部地点\S*',t).end()]
        if re.search('所属行业',t):
            item["industry_b"]=t[re.search('所属行业',t).end()+1:re.search('所属行业\S*',t).end()]
        if re.search('公司福利',t):
            item["welfare"]=t[re.search('公司福利',t).end()+1:re.search('公司福利\S*',t).end()]
        if soup.find('div',class_="gszc"):
            for t in soup.find('div',class_="gszc").div.find_all('td',class_="tr-title"):
                item[t.text[:-1]]=t.next_sibling.next_sibling.text
        t=soup.find('div',class_="qynb").find_all('td')
        for tdn in range(len(t)):
            if t[tdn].text=='电子邮箱：':
                item["电子邮箱"]=t[tdn+1].text
            elif t[tdn].text=='企业联系电话：':
                item["企业联系电话"]=t[tdn+1].text

        for t in soup.find('div',class_="gszc").find_all('h2')[1:]:
            if t.text=='主要人员':
                st=''
                for td in t.next_sibling.next_sibling.find_all('td',class_="tr-title"):
                    st+=td.text+td.next_sibling.next_sibling.text+';'
                item["主要人员"]=st[:-1]
                    
            elif t.text=='股东信息':
                st=''
                if len(t.next_sibling.next_sibling.find_all('tr'))>1:
                    for tr in t.next_sibling.next_sibling.find_all('tr')[1:]:
                        for td in tr.find_all('td'):
                            st+=td.text+','
                        st=st[:-1]+';'
                item["股东信息"]=st[:-1]
        st=''
        for a in soup.find('div',class_="position").find_all('a'):
            if not a.span:
                break
            st+=a.span.text+','
            for p in a.find_all('p'):
                st+=p.text+','
            st=st[:-1]+';'
        item["热招职位"]=st[:-1]
        
        

        try:
            cursor.execute(sql,(item['url'],item['company'],item['province'],item['website'],item['industry_b'],item['headquarters'],item['welfare'],item['工商注册号'],item['组织机构代码'],item['统一信用代码'],item['经营状态'],item['行业'],item['企业类型'],item['法定代表人'],item['营业期限'],item['注册资本'],item['核准日期'],item['登记机关'],item['企业地址'],item['经营范围'],item['企业联系电话'],item['电子邮箱'],item['主要人员'],item['股东信息'],item['热招职位']))
            dbObject.commit()
            #print('写入成功')
        except Exception as e:
            print(e)
            print('###########写入失败！############')
            dbObject.rollback()
            
        
        
        '''
        djson={}
        if soup.find('table',class_='gdxx'):
            djson['title']=[t.text for t in soup.find('table',class_='gdxx').find('tr').find_all('th')]
            djson['list']=[]
            for tlist in soup.find('table',class_='gdxx').find_all('tr')[1:]:
                djson['list'].append([t.text for t in tlist.find_all('td')])
        df.loc[n,'gdxx']=json.dumps(djson)
        '''
    
    n+=1
print('完成！')

In [16]:
all_path[137785]

'a78e151884335688f0fe09a87e7ac231.txt'